In [ ]:
1`1 !rm -rf /content/*
!pip install git+https://github.com/openai/whisper.git 
!sudo apt update && sudo apt install ffmpeg
!pip install --upgrade --no-cache-dir gdown
# https://drive.google.com/file/d/1akD8y6G6pB5YpMcvzPP74yK00kWIa4P3/view?usp=sharing
!gdown https://drive.google.com/uc?id=1akD8y6G6pB5YpMcvzPP74yK00kWIa4P3
!unzip /content/Downloads.zip
!rm -rf /content/Downloads.zip
!git clone https://github.com/x4nth055/gender-recognition-by-voice.git
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install -r /content/gender-recognition-by-voice/requirements.txt
!pip install streamlit

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/de

In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
import whisper
import torch
import os
import warnings
warnings.filterwarnings("ignore")

# Initialize the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model 
whisper_model = whisper.load_model("large", device=device)

100%|█████████████████████████████████████| 2.87G/2.87G [01:01<00:00, 50.3MiB/s]


**Language Detection**

In [ ]:
# load audio and pad/trim it to fit 30 seconds
import IPython.display as ipd
from IPython.core.display import display

display(ipd.Audio("/content/french.mp3"))
audio = whisper.load_audio("/content/french.mp3")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(whisper_model.device)

# detect the spoken language
_, probs = whisper_model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

Detected language: fr


In [ ]:
# load audio and pad/trim it to fit 30 seconds
display(ipd.Audio("/content/english.wav"))
audio = whisper.load_audio("/content/english.wav")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(whisper_model.device)

# detect the spoken language
_, probs = whisper_model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

Detected language: en


**Language Transalation to english**

In [ ]:
display(ipd.Audio("/content/english.wav"))
result = whisper_model.transcribe("/content/english.wav")
print(result["text"])

 Kids are talking by the door.


In [ ]:
display(ipd.Audio("/content/french.mp3"))
result = whisper_model.transcribe("/content/french.mp3")
print(result["text"])

 Comment t'appelles-tu ?


In [ ]:
display(ipd.Audio("/content/french.mp3"))
audio_file = "/content/french.mp3"
result = whisper_model.transcribe(audio_file, task = 'translate')
print(result["text"])

 What's your name?


**Gender detection**

In [ ]:
%cd /content/gender-recognition-by-voice

/content/gender-recognition-by-voice


In [ ]:
%%writefile train.py
import os
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

from utils import load_data, split_data, create_model

# load the dataset
X, y = load_data()
# split the data into training, validation and testing sets
data = split_data(X, y, test_size=0.1, valid_size=0.1)
# construct the model
model = create_model()

# use tensorboard to view metrics
tensorboard = TensorBoard(log_dir="logs")
# define early stopping to stop training after 5 epochs of not improving
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)

batch_size = 64
epochs = 10

# train the model using the training set and validating using validation set
model.fit(data["X_train"], data["y_train"], epochs=epochs, batch_size=batch_size, validation_data=(data["X_valid"], data["y_valid"]),
          callbacks=[tensorboard, early_stopping])

# save the model to a file
model.save("results/model.h5")

# evaluating the model using the testing set
print(f"Evaluating the model using {len(data['X_test'])} samples...")
loss, accuracy = model.evaluate(data["X_test"], data["y_test"], verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Overwriting train.py


In [ ]:
!python train.py

2023-05-06 03:44:50.549874: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 03:44:54.021518: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               33024     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Drop

In [ ]:
import librosa
import numpy as np
import tensorflow as tf

def extract_feature(file_name, **kwargs):
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    X, sample_rate = librosa.core.load(file_name)
    if chroma or contrast:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
    if contrast:
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, contrast))
    if tonnetz:
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
        result = np.hstack((result, tonnetz))
    return result

def get_gender_prediction(file_path,model):
    features = extract_feature(file_path, mel=True).reshape(1, -1)
    male_prob = model.predict(features)[0][0]
    female_prob = 1 - male_prob
    gender = "male" if male_prob > female_prob else "female"
    if gender == 'male':
        return gender,round(male_prob*100,2)
    else:
        return gender,round(female_prob*100,2)

In [ ]:
model_path = '/content/gender-recognition-by-voice/results/model.h5'
file_path = '/content/female.wav'
display(ipd.Audio(file_path))
model = tf.keras.models.load_model(model_path)
pred,prob_score = get_gender_prediction(file_path,model)
pred,prob_score

ValueError: ignored

In [ ]:
model_path = '/content/gender-recognition-by-voice/results/model.h5'
file_path = '/content/male.wav'
display(ipd.Audio(file_path))
pred,prob_score = get_gender_prediction(file_path,model)
pred,prob_score

1/1 [==============================] - 0s 22ms/step


('male', 84.66)